In [1]:
import pandas as pd
import random

In [ ]:
persona_path = r"C:\Users\aman\Desktop\v1 dataset\rawdata\PersonaChat\personality.csv"
df = pd.read_csv(persona_path)

In [4]:

def convert_to_compatika(user_msg):
    templates = [
        "It’s nice you reached out. How are you feeling right now?",
        "Thanks for sharing that. What’s been on your mind lately?",
        "I hear you. What emotions are coming up for you today?",
        "It sounds like there’s a lot underneath. Want to talk about it?"
    ]
    return random.choice(templates)

In [5]:
compatika_data = []

for i, row in df.iterrows():
    user_msg = row["chat"]
    compatika_reply = convert_to_compatika(user_msg)

    compatika_data.append({
        "USER": user_msg,
        "COMPATIKA": compatika_reply
    })

final_df = pd.DataFrame(compatika_data)
final_df.to_csv("persona_fixed_compatika.csv", index=False)


print("Fixed dataset saved.")

Fixed dataset saved.


In [12]:
import pandas as pd
import random
import os

# -------- CONFIG --------
persona_path = r"C:\Users\aman\Desktop\v1 dataset\rawdata\PersonaChat\personality.csv"
output_txt = "persona_compatika_dataset.txt"

# -------- Helper functions --------
def find_text_column(df):
    """Auto-detect which column contains chat/user text."""
    candidates = ["USER", "user", "chat", "text", "utterance", "message"]
    cols = [c.lower() for c in df.columns]
    
    for cand in candidates:
        if cand.lower() in cols:
            return df.columns[cols.index(cand.lower())]
    
    # fallback: longest text column
    text_cols = [c for c in df.columns if df[c].dtype == object]
    if text_cols:
        avg_lengths = {c: df[c].astype(str).map(len).mean() for c in text_cols}
        return max(avg_lengths, key=avg_lengths.get)
    
    return None

def split_lines(text):
    """Split long multi-turn persona chats into separate lines."""
    if pd.isna(text):
        return []
    cleaned = []
    for line in str(text).replace("\r", "\n").split("\n"):
        line = line.strip()
        if len(line) > 3:
            cleaned.append(line)
    return cleaned

# Compatika gentle default replies
TEMPLATES = [
    "It’s nice you reached out. How are you feeling right now?",
    "Thanks for sharing that. What’s been on your mind lately?",
    "I’m here with you. What emotions are coming up for you now?",
    "That sounds meaningful. How are you feeling as you say that?",
    "I hear you. Want to tell me more about what you're feeling?"
]

# -------- LOAD FILE --------
if not os.path.exists(persona_path):
    raise FileNotFoundError(f"File not found: {persona_path}")

df = pd.read_csv(persona_path, dtype=str, keep_default_na=False)
print("Loaded columns:", df.columns.tolist())

text_col = find_text_column(df)
print("Detected text column:", text_col)

if text_col is None:
    raise ValueError("No valid text column detected!")

# -------- GENERATE TXT DATA --------
with open(output_txt, "w", encoding="utf-8") as out:
    for _, row in df.iterrows():
        messages = split_lines(row[text_col])
        
        for msg in messages:
            reply = random.choice(TEMPLATES)
            
            out.write(f"USER: {msg}\n")
            out.write(f"COMPATIKA: {reply}\n\n")  # blank line between samples

print(f"TXT dataset generated successfully: {output_txt}")


Loaded columns: ['Unnamed: 0', 'Persona', 'chat']
Detected text column: chat
TXT dataset generated successfully: persona_compatika_dataset.txt


Wrote 473080 samples to compatika_cleaned.txt

--- Preview (first 10) ---

USER: hi , how are you doing ?
COMPATIKA: Thanks for sharing that. What’s been on your mind lately?

USER: i am getting ready to do some sprinting to stay in shape .
COMPATIKA: That sounds meaningful. How are you feeling as you say that?

USER: COMPATIKA: That sounds meaningful.
COMPATIKA: I hear you. Want to tell me more about how that feels?

USER: How are you feeling as you say that?
COMPATIKA: I hear you. Want to tell me more about how that feels?

USER: you must be very fast .
COMPATIKA: Thanks for sharing that. What’s been on your mind lately?

USER: hunting is one of my favorite hobbies .
COMPATIKA: Thanks for sharing that. What’s been on your mind lately?

USER: COMPATIKA: Thanks for sharing that.
COMPATIKA: That sounds meaningful. How are you feeling as you say that?

USER: What’s been on your mind lately?
COMPATIKA: Thanks for sharing that. What’s been on your mind lately?

USER: i am !
COMPATIKA: I he